In [2]:
import PyPDF2
import json
import glob
import numpy as np
import pdfplumber
import pandas as pd

from itertools import chain
from collections import defaultdict
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure

In [3]:
def monthToNum(shortMonth):
    return {
            'January': "01",
            'February': "02",
            'March': "03",
            'April': "04",
            'May': "05",
            'June': "06",
            'July': "07",
            'August': "08",
            'September': "09", 
            'October': "10",
            'November': "11",
            'December': "12"
    }[shortMonth]

def defaultify(d):
        
    if not isinstance(d, dict):
        return d
    return defaultdict(lambda: None, {k: defaultify(v) for k, v in d.items()})

In [4]:
path_output=\
'/Volumes/4TB/ConferenceCall/Tesla/csvfile/tesla_earning_call_question_answer.json'

In [5]:
with open(path_output, 'r') as openfile:
     json_object = json.load(openfile)

In [29]:
def_object=defaultify(json_object)

In [48]:
data_table=\
pd.DataFrame([
              col1, col2, col3, d3
             ]
              for col1, d1 in def_object.items()\
              for col2, d2 in d1.items()\
              for col3, d3 in d2.items()
            )
data_table=data_table.rename(columns={0: 'time',
                                      1: 'company',
                                      2: 'chara1',
                                      3: 'text'}).reset_index(drop=True)

In [55]:
data_table.head()

,time,company,chara1,text
0,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",people,{'analysts': {'Adam Michael Jonas': 'Morgan St...
1,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",question_and_answer,"[Question and Answer\n, Operator\n, [Operator ..."
2,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",executives_answer,"{'answer_0': {'6': ['Sure, our goal with Tesla..."
3,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",analysts_question,{'question_0': {'4': ['So I guess a couple que...
4,"Wednesday, May 04, 2011 9:00 PM GMT\nS&P","Tesla Motors, Inc., Q1 2011 Earnings Call, May...",executives_people_answer,"{'answer_0': 'Elon R. Musk', 'answer_1': 'Elon..."


In [209]:
data_table_01=data_table[data_table['chara1'] == 'executives_answer'].reset_index(drop=True)

data_table_01['text_executives_answer_01']=\
data_table_01.loc[:, 'text'].apply(
    lambda x: list(x.values()))

data_table_01['text_executives_answer_02']=\
data_table_01['text_executives_answer_01'].apply(lambda x: [list(text.values())[0][0] for text in x])

data_table_01['text_executives_answer_02']=\
data_table_01['text_executives_answer_02'].apply(lambda x: " ".join([text for text in x]))

data_table_01=data_table_01[['time', 'company', 'chara1', 'text_executives_answer_02']]

data_table_01=\
pd.pivot(data_table_01,
         values=['text_executives_answer_02'],
         index=['time', 'company'],
         columns=['chara1'])

data_table_01=data_table_01['text_executives_answer_02'].reset_index(drop=False)

In [210]:
data_table_01["call_date"]=data_table_01["time"].apply(lambda x: x.split(',')[1])
data_table_01["call_year"]=\
data_table_01["time"].apply(lambda x: x.split(',')[2][0:5])
data_table_01["call_time"]=\
data_table_01["time"].apply(lambda x: x.split(',')[2][5:11])
data_table_01["quarter"]=\
data_table_01["company"].apply(lambda x: x.split(',')[2][:3])
data_table_01["year_quarter"]=\
data_table_01["company"].apply(lambda x: x.split(',')[2][3:9])
data_table_01["company_name"]=data_table_01["company"].apply(lambda x: x.split(',')[0][0:12])
data_table_01=data_table_01.sort_values(['year_quarter', 'quarter'])
data_table_01=data_table_01.reset_index(drop=True)

data_table_01['call_month']=\
data_table_01['call_date'].apply(
    lambda x: x.split(' ')[1]).apply(
    lambda x: monthToNum(x))

data_table_01['call_day']=\
data_table_01['call_date'].apply(
    lambda x: x.split(' ')[2])

cols = data_table_01.loc[:, ['call_year', 'call_month', 'call_day']].columns
data_table_01['call_datetime']=\
data_table_01[cols].apply(
    lambda x: '-'.join(x.dropna().astype(str)), 1)

data_table_01=\
data_table_01[['time',
                'call_datetime',
                'call_year',
                'call_month',
                'call_day',
                'company_name',
                'company',
                'quarter',
                'year_quarter',
                'executives_answer']]

In [211]:
path_output=\
'/Volumes/4TB/ConferenceCall/Tesla/csvfile/tesla_earning_call_executives_answer.csv'

data_table_01.to_csv(path_output, index = False)

In [212]:
data_table_02=data_table[data_table['chara1'] == 'analysts_question'].reset_index(drop=True)

data_table_02['text_analysts_question_01']=\
data_table_02.loc[:, 'text'].apply(
    lambda x: list(x.values()))

data_table_02['text_analysts_question_02']=\
data_table_02['text_analysts_question_01'].apply(lambda x: [list(text.values())[0][0] for text in x])

data_table_02['text_analysts_question_02']=\
data_table_02['text_analysts_question_02'].apply(lambda x: " ".join([text for text in x]))

In [213]:
data_table_02=data_table_02[['time', 'company', 'chara1', 'text_analysts_question_02']]

data_table_02=\
pd.pivot(data_table_02,
         values=['text_analysts_question_02'],
         index=['time', 'company'],
         columns=['chara1'])

data_table_02=data_table_02['text_analysts_question_02'].reset_index(drop=False)

data_table_02["call_date"]=data_table_02["time"].apply(lambda x: x.split(',')[1])
data_table_02["call_year"]=\
data_table_02["time"].apply(lambda x: x.split(',')[2][0:5])
data_table_02["call_time"]=\
data_table_02["time"].apply(lambda x: x.split(',')[2][5:11])
data_table_02["quarter"]=\
data_table_02["company"].apply(lambda x: x.split(',')[2][:3])
data_table_02["year_quarter"]=\
data_table_02["company"].apply(lambda x: x.split(',')[2][3:9])
data_table_02["company_name"]=data_table_02["company"].apply(lambda x: x.split(',')[0][0:12])
data_table_02=data_table_02.sort_values(['year_quarter', 'quarter'])
data_table_02=data_table_02.reset_index(drop=True)

data_table_02['call_month']=\
data_table_02['call_date'].apply(
    lambda x: x.split(' ')[1]).apply(
    lambda x: monthToNum(x))

data_table_02['call_day']=\
data_table_02['call_date'].apply(
    lambda x: x.split(' ')[2])

cols = data_table_02.loc[:, ['call_year', 'call_month', 'call_day']].columns
data_table_02['call_datetime']=\
data_table_02[cols].apply(
    lambda x: '-'.join(x.dropna().astype(str)), 1)

data_table_02=\
data_table_02[['time',
                'call_datetime',
                'call_year',
                'call_month',
                'call_day',
                'company_name',
                'company',
                'quarter',
                'year_quarter',
                'analysts_question']]

In [214]:
path_output=\
'/Volumes/4TB/ConferenceCall/Tesla/csvfile/tesla_earning_call_analysts_question.csv'

data_table_02.to_csv(path_output, index = False)